In [7]:
import pandas as pd

df = pd.read_csv('out.csv')

In [8]:
for i, cont in enumerate(df['Context']):
    df.at[i, 'Context'] = (cont.split('/')[1].split('.')[0])

In [9]:
df

,Context,Epsilon,Delta,Aproximation_type,Distribution_type,Threads,Total_execution_time,Total_time,Total_exec_time_2,Total_closure_time,...,Basis_size,Total_counter_examples,Sum_total_tries,Equal_to_count,Empty_set_closure_computes,Support_avg,Support_p10,Support_p50,Support_p90,Support_p95
0,connect,0.01,0.1,weak,area-based,16,0.268294,0.114895,0.069117,0.0,...,16,67,11075,154,1,87.546732,99.986153,96.981446,63.818887,10.800332
1,connect,0.01,0.1,weak,area-based,16,0.211694,0.073410,0.058552,0.0,...,15,63,6267,162,1,84.901689,99.986153,95.042924,72.694544,0.083079
2,connect,0.01,0.1,weak,area-based,16,0.341495,0.140697,0.093164,0.0,...,19,93,12720,426,1,90.106254,99.986153,99.099972,72.680698,0.069233
3,connect,0.01,0.1,weak,area-based,16,0.244911,0.079046,0.062936,0.0,...,16,74,6930,291,1,89.081972,99.986153,99.210745,74.729992,18.055940
4,connect,0.01,0.1,weak,area-based,16,0.341777,0.152923,0.100228,0.0,...,20,94,14266,422,1,87.756854,99.986153,95.098311,53.406259,50.900028
5,connect,0.01,0.1,weak,area-based,16,0.346958,0.156494,0.106455,0.0,...,20,99,13949,464,1,88.701883,99.986153,99.099972,60.509554,0.969261
6,connect,0.01,0.1,weak,area-based,16,0.228433,0.084517,0.063026,0.0,...,15,63,7752,146,1,90.471707,99.986153,99.210745,64.580449,40.708945
7,connect,0.01,0.1,weak,area-based,16,0.340885,0.136964,0.098162,0.0,...,19,94,12948,404,1,88.558352,99.986153,99.210745,49.016893,27.540847
8,connect,0.01,0.1,weak,area-based,16,0.257270,0.085315,0.070205,0.0,...,17,74,7777,301,1,95.736882,99.986153,99.183052,87.621158,68.346718
